# 取引履歴のデータ分析

In [23]:
import os
import pandas as pd

In [24]:
current_dir = %pwd

os.path.dirname(current_dir)

'/home/tsu-nera/repo'

In [3]:
import sys
sys.path.append('/home/tsu-nera/repo/bakuchi/')
%cd '../../'

/home/tsu-nera/repo/bakuchi


In [4]:
import src.utils.trade_history as th

%reload_ext autoreload
%autoreload 2

File doesn't exist 


In [5]:
timestamp="2003182338"

In [6]:
cc_trades = th.read_coincheck(timestamp)
lq_trades = th.read_liquid(timestamp)

In [7]:
len(cc_trades), len(lq_trades)

(164, 164)

In [8]:
profit_df = pd.DataFrame(
    {'timestamp': cc_trades['datetime'].to_list(),
     'cc_side': cc_trades['side'].to_list(),
     'cc_price': cc_trades['price'].to_list(), 
     'lq_side': lq_trades['side'].to_list(),
     'lq_price': lq_trades['price'].to_list()
    }
)

def calc_profit(x):
    if x['cc_side'] == 'sell':
        cc_price = x['cc_price']
    else:
        cc_price = -1 * x['cc_price']
        
    if x['lq_side'] == 'sell':
        lq_price = x['lq_price']
    else:
        lq_price = -1 * x['lq_price']

    return cc_price + lq_price

profit_df['profit'] = profit_df.apply(calc_profit, axis=1)

profit_df.timestamp=pd.to_datetime(profit_df.timestamp,format='%Y%m%d %H%M%S')
profit_df = profit_df.set_index('timestamp')

In [9]:
profit_df

,cc_side,cc_price,lq_side,lq_price,profit
timestamp,,,,,
2020-03-18 03:19:25,sell,5260.216,buy,5257.620,2.596
2020-03-18 03:19:45,buy,5262.997,sell,5260.878,-2.119
2020-03-18 03:21:23,sell,5256.963,buy,5254.722,2.241
2020-03-18 03:23:00,buy,5237.998,sell,5237.883,-0.115
2020-03-18 03:23:12,sell,5234.256,buy,5227.668,6.588
...,...,...,...,...,...
2020-03-18 09:02:11,buy,5225.997,sell,5226.822,0.825
2020-03-18 09:02:34,sell,5238.018,buy,5233.968,4.050
2020-03-18 09:02:42,buy,5243.997,sell,5240.034,-3.963


In [10]:
round(sum(profit_df.profit), 3)

180.299

## 資産の考慮

In [11]:
asset = {
  "coincheck": {
    "start_jpy": 5496,
    "start_btc": 0.010948,
    "end_jpy": 5843,
    "end_btc": 0.010038
  },
  "liquid": {
    "start_jpy": 5910,
    "start_btc": 0.11,
    "end_jpy": 5744,
    "end_btc": 0.011
  },
  "total": {
    "start_jpy": 11406,
    "start_btc": 0.021948,
    "end_jpy": 11587,
    "end_btc": 0.021038
  }
}

In [12]:
total = asset['total']
total_profit_jpy = round(total['end_jpy'] - total['start_jpy'], 3)
total_profit_btc = round(total['end_btc'] - total['start_btc'], 3)

In [13]:
start_bid = 583922.0
end_bid = 580140.0

In [14]:
total_profit_jpy +  total_profit_btc * ((end_bid + start_bid)/2)

-401.03100000000006

In [15]:
actual_start_price = round(total['start_jpy'] + total['start_btc'] * start_bid, 3)
actual_end_price = round(total['end_jpy'] + total['end_btc'] * end_bid, 3)
actual_profit = actual_end_price - actual_start_price

In [16]:
notrade_start_price = actual_start_price
notrade_end_price = round(total['start_jpy'] + total['start_btc'] * end_bid, 3)
notrade_profit = notrade_end_price - notrade_start_price

In [17]:
trading_profit = actual_profit - notrade_profit
time_profit = notrade_profit

In [18]:
int(actual_profit), int(time_profit), int(trading_profit)

(-429, -83, -346)